## Data Version Control (DVC) exercise - creating a pipeline

This exercise will use DVC to capture a data pipeline recreating the steps performed in training a periodic spline model on historic temperature range measurements in [_Lecture 22 (Data Pipelines)_](Lecture22_DataPipelines.ipynb). This exercise will assume you have some familiarity with using Git from a command-line. If you are not familiar with the Git command line interface you can run just the DVC commands without also tracking the changes with Git - in this case you will need [to pass the `--no-scm` option to `dvc init`](https://dvc.org/doc/command-reference/init#initializing-dvc-without-git) and ignore the exercise parts <span style="color: red;"> in red</span>.

Running the cell below creates a new `dvc-pipeline-example` directory in the current working directory and changes this to the working directory. This ensures the files we create to construct the example pipeline will be kept isolated.

In [ ]:
import os
os.mkdir("dvc-pipeline-example")
os.chdir("dvc-pipeline-example")

To use DVC you will need to have [DVC installed](https://dvc.org/doc/install) and to be able to run the `dvc` command from a [shell](https://en.wikipedia.org/wiki/Shell_(computing)) (command-line interface to the operating system). <span style="color: red;">You will also need to have Git installed and be able to run `git` for the Git parts of the exercises.</span> If you are running the Jupyter notebook server from a Linux or MacOS system (or from a [Windows Subsystem for Linux](https://learn.microsoft.com/en-us/windows/wsl/install) shell on Windows) then you can use the [`%%sh`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-bash) IPython cell magic command to run shell commands directly from cells in the notebook - just write `%%sh` as the first line of the cell followed by one or more lines corresponding to the commands to execute. On Windows, you can launch an Anaconda Prompt terminal and run the commands from there (ensuring `dvc` and `git` are installed in the activated `conda` environment and that you are within the `dvc-pipeline-example` directory).

## Initialising Git and DVC

As a first step 
  1. <span style="color: red;">Create a new empty Git repository in the `dvc-pipeline-example` directory.</span>
  2. Initialise a new DVC project in the `dvc-pipeline-example` directory.
  3. <span style="color: red;">Commit the files created by DVC to the current Git branch.</span>

## Importing the remote data file

  1. Use the `dvc import-url` command to import the gzipped CSV file `1800.csv.gz` containing the historical climate records for the year 1800 from the `noaa-ghcn-pds` Amazon Web Services (AWS) S3 bucket in to the local project.
  2. <span style="color: red;">Add the files generated by DVC on importing the data file to the staging area and commit these changes with Git.</span>
  
_Hint_: To avoid having provide Amazon Web Service credentials (which are required when using DVC's built in support for accesing AWS S3 storage) you can instead use HTTP to access data on the bucket using a URL of the form `http://{bucket-name}.s3.amazonaws.com/{path-to-file}` where `{bucket-name}` is the name of the (publicly accessible) S3 bucket to access and `{path-to-file}` is the path to the file to access on the bucket.

## Writing a Python script to prepare data for model training

Create a Python script `prepare.py` in the `dvc-pipeline-example` directory which recreates the preprocess, transform and serve stages of the example pipeline the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb). That is the script should

  1. Extract the CSV data from the downloaded `1800.csv.gz` file.
  2. Select the records corresponding to temperature (`TMIN` and `TMAX`) measurements for station `ITE00100554`.
  3. Transform the extreme temperature measurements to temperature ranges in degrees Celsius.
  4. Write out the transformed data as a feature matrix (containing the day of year of the measurement as integers) and targets array (containing the temperature ranges in degrees Celsius) ready to feed in to a scikit-learn regression model.

The script should take as arguments the path to the input data file and the path to write the prepared training data to.

_Hint 1_: Most of the code you need is already in the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb).  
_Hint 2_: The [`numpy.savez`](https://numpy.org/doc/stable/reference/generated/numpy.savez.html) function may be useful for writing NumPy array data to a file.  
_Hint 3_: You can use [the `%%writefile` cell magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#cellmagic-writefile) to create a Python script file from the contents of a cell within the notebook.

## Adding data preparation pipeline stage to DVC

Add a stage to the DVC data pipeline corresponding to preparing the data for model training using the script you just created. The stage should

  * Be named `prepare`.
  * Have as dependencies the `prepare.py` script and the `1800.csv.gz` data file.
  * Have an output file containing the prepared training data feature matrix and target array.
  * Execute the `prepare.py` script using `python` as the pipeline command, passing in the input data file and output file name as arguments.

You can add the stage either by directly creating a `dvc.yaml` file or using the `dvc stage add` command. <span style="color: red;">Once you have created the stage add the relevant files for the stage to the Git staging area and create a new commit.</span>

## Writing a Python script to train scikit-learn regression model

Create a Python script `train.py` in the `dvc-example-pipeline` directory which recreates the model training stage of the example pipeline the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb). That is the script should

  1. Create scikit-learn pipeline corresponding to [a periodic spline regression model with L2 regularization](https://scikit-learn.org/stable/auto_examples/linear_model/plot_polynomial_interpolation.html#periodic-splines).
  2. Fit the model to the training data feature matrix and targets array prepared by the previous pipeline stage.
  3. Serialize the trained model to a file.
  
The number of knots to use in the spline model, degree of the spline polynomial and [L2 regularization term $\alpha$ coefficient](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) should all be configurable parameters read from an external file (for example in JSON or YAML format) to allow them to be [captured as parameters of the pipeline stage](https://dvc.org/doc/start/data-management/metrics-parameters-plots#defining-stage-parameters). The script should accept as arguments the path to the file containing the prepared training data (output of the previous pipeline stage), the path to the parameters file and the path to output a serialization of the trained model to.

_Hint 1_: Most of the code you need is again in the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb).  
_Hint 2_: The [`json` module in the Python standard library](https://docs.python.org/3/library/json.html) or [third-party PyYAML package](https://pyyaml.org/wiki/PyYAMLDocumentation) may be useful for reading from the parameter file.

## Adding a model training pipeline stage

Add a stage to the DVC data pipeline corresponding to training the model using the script you just created. The stage should

  * Be named `train`.
  * Have as parameters the number of spline knots, spline polynomial degree and L2 regularization coefficient $\alpha$.
  * Have as dependencies the `train.py` script and training data file outputted by the previous `prepare` stage.
  * Have an output file corresponding to the serialized trained model.
  * Execute the `train.py` script using `python` as the pipeline command, passing in the training data file, parameters file and output file name as arguments.
  
You will also need to create the parameter file - for the parameter values we suggest using 7 spline knots, a spline degree of 3 and $\alpha$ L2 coefficient value of 0.01. <span style="color: red;">Once you have created the stage add the relevant files for the stage to the Git staging area and create a new commit.</span>

## Writing a Python script to plot model predictions

Create a Python script `plot.py` in the `dvc-example-pipeline` directory which plots the trained model's predictions as in the example pipeline the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb). That is the script should

  1. Load the serialized train model and training data from files.
  2. Compute the model predictions on the training data input features.
  3. Plot the model predictions and training target values (daily temperature range in degrees Celsius) against the corresponding input feature values (day of the year) as separate line plots on the same axes using Matplotib.
  4. Save the generated plot figure to a file.

The script should accept as arguments the path to the file containing the  training data, the path to the file containing the serialized trained model and the path to output the saved figure to.

_Hint 1_: Most of the code you need is again in the [_Lecture 22 (Data Pipelines)_ notebook](Lecture22_DataPipelines.ipynb).  
_Hint 2_: The [Matplotlib `Figure.savefig` method](https://matplotlib.org/stable/api/figure_api.html#matplotlib.figure.Figure.savefig) may be useful for saving the plot figure to a file.

## Adding a plotting pipeline stage

Add a stage to the DVC data pipeline corresponding to plotting the model predictions using the script you just created. The stage should

  * Be named `plot`.
  * Have as dependencies the `plot.py` script, and training data and serialized model files outputted by the previous stages.
  * Have an output file corresponding to the generated figure.
  * Execute the `plot.py` script using `python` as the pipeline command, passing in the training data file, trained model file and output file name as arguments.
  
<span style="color: red;">Once you have created the stage add the relevant files for the stage to the Git staging area and create a new commit.</span>

## Visualizing and running the pipeline

Visualize the DVC data pipeline you have created as a directed acyclic graph and test running the pipeline.


_Hint_: The [DVC Data Pipelines documentation](https://dvc.org/doc/start/data-management/data-pipelines) explains the commands you need here.